In [73]:
import json
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer


In [ ]:
APIKEY = "YOUR_API_KEY"

In [55]:
pc = Pinecone(api_key=APIKEY)
index = pc.Index("healthcare-chatbot")

model = SentenceTransformer('all-MiniLM-L6-v2')

/home/fm-pc-lt-237/miniconda3/envs/llm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [36]:
#sample datas
# datas = [
#     {
#         "id": "anisocytosis_overview_1",
#         "metadata": {
#             "title": "What is anisocytosis?",
#             "content": "Anisocytosis (pronounced “a-nuh-soe-sai-TOW-suhs”) describes red blood cells that vary in size. Sometimes, size variation in your red blood cells may signal an underlying medical issue. You may learn that your red blood cells are different sizes when you receive results from a blood test.",
#             "source": "https://my.clevelandclinic.org/health/diseases/24997-anisocytosis#overview",
#         },
#         "values": [.11]*384,
#     },
#     {
#         "id": "anisocytosis_symptoms_1",
#         "metadata": {
#             "title": "Symptoms of anisocytosis?",
#             "content": "Anisocytosis (pronounced “a-nuh-soe-sai-TOW-suhs”) describes red blood cells that vary in size. Sometimes, size variation in your red blood cells may signal an underlying medical issue. You may learn that your red blood cells are different sizes when you receive results from a blood test.",
#             "source": "https://my.clevelandclinic.org/health/diseases/24997-anisocytosis#symptoms-and-causes",
#         },
#         "values": [.12]*384
#     },
# ]

In [54]:
# !pip install sentence-transformers

In [56]:

with open('dataset.json', 'r') as json_file:
    dataset = json.load(json_file)

In [72]:
import os
for filename in os.listdir("sample_dataset"):
    if filename.endswith('.json'):
        file_path = os.path.join("sample_dataset", filename)
        with open(file_path, 'r') as json_file:
            dataset = json.load(json_file)
            # Do something with the dataset
            print(f"{filename} reading...")
            
            for disease_name in dataset:
                print(f"{disease_name} indexing...", end='\r')  # Print indexing message and overwrite on the same line
                for j in dataset[disease_name]:
                    for idx, question in enumerate(dataset[disease_name][j]):
                        if question == "source":
                            continue
                        data = {}
                        data["id"] = f"{disease_name}_{j}{idx}"
                        data["values"] = model.encode(dataset[disease_name][j][question]).tolist()
                        data["metadata"] = {}
                        data["metadata"]["title"] = question
                        data["metadata"]["content"] = dataset[disease_name][j][question]
                        data["metadata"]["source"] = dataset[disease_name][j]["source"]
                        
                        #index pinecone
                        try:
                            index.upsert(
                                [data]
                            )
                        except Exception as e:
                            print("Error while indexing")
                    
                print(f"{disease_name} indexed!")

dataset_b.json reading...
Babesiosis indexed!...
Baby Acne indexed!...
dataset_a.json reading...
AA Amyloidosis indexed!...
Abdominal Adhesions indexed!...


In [57]:

with open('dataset.json', 'r') as json_file:
    dataset = json.load(json_file)

for disease_name in dataset:
    for j in dataset[disease_name]:
        for idx, question in enumerate(dataset[disease_name][j]):
            if question == "source":
                continue
            data = {}
            data["id"] = f"{disease_name}_{j}{idx}"
            data["values"] = model.encode(dataset[disease_name][j][question]).tolist()
            data["metadata"] = {}
            data["metadata"]["title"] = question
            data["metadata"]["content"] = dataset[disease_name][j][question]
            data["metadata"]["source"] = dataset[disease_name][j]["source"]
            
            #index pinecone
            try:
                index.upsert(
                    [data]
                )
            except Exception as e:
                print("Error while indexing")
            
            print(f"{disease_name} indexed...")
        # break
    # break

{'id': 'Babesiosis_overview1', 'values': [-0.01822429522871971, -0.030677618458867073, -0.012573560699820518, 0.025656163692474365, -0.0225053820759058, 0.022542258724570274, 0.015020569786429405, 0.07880654186010361, -0.021252742037177086, 0.06178835779428482, 0.004030147101730108, -0.05446769297122955, -0.04514389857649803, 0.08581119775772095, -0.0006713197217322886, -0.0016847571823745966, -0.029650229960680008, -0.07541090995073318, -0.03032032772898674, -0.02500440925359726, -0.06579533964395523, 0.030194245278835297, -0.0416032075881958, -0.009405974298715591, -0.011868083849549294, -0.03182263299822807, 0.042568389326334, -0.00816420093178749, 0.0008465991704724729, -0.002664639614522457, -0.07099708169698715, -0.05081155523657799, -0.029827497899532318, -0.037518687546253204, 0.05137128010392189, 0.018296897411346436, 0.037684377282857895, -0.060413211584091187, 0.04834712669253349, 0.012387458235025406, -0.046202562749385834, -0.010934612713754177, 0.028624150902032852, -0.01

In [61]:
query = "Describe a disease called Babesosis and how to avoid getting it."
query_vector = model.encode(query).tolist()
index.query(
    # namespace="example-namespace",
    vector=query_vector,
    top_k=3,
    # include_values=True
)

{'matches': [{'id': 'Babesiosis_overview1', 'score': 0.714829087, 'values': []},
             {'id': 'Babesiosis_symptoms-and-causes2',
              'score': 0.655548155,
              'values': []},
             {'id': 'Babesiosis_symptoms-and-causes4',
              'score': 0.630927086,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [ ]:
import os
import json
import pinecone
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

DATASET_DIR = "sample_dataset"
APIKEY = ""
#pinecone
pc = Pinecone(api_key=APIKEY)
index = pc.Index("healthcare-chatbot")

#sentence transformer
model = SentenceTransformer('all-MiniLM-L6-v2')

def main():
    for filename in os.listdir(DATASET_DIR):
        if filename.endswith('.json'):
            file_path = os.path.join(DATASET_DIR, filename)
            with open(file_path, 'r') as json_file:
                dataset = json.load(json_file)
                # Do something with the dataset
                print(f"{filename} reading...")
                
                for disease_name in dataset:
                    print(f"{disease_name} indexing...", end='\r')  # Print indexing message and overwrite on the same line
                    for j in dataset[disease_name]:
                        for idx, question in enumerate(dataset[disease_name][j]):
                            if question == "source":
                                continue
                            data = {}
                            data["id"] = f"{disease_name}_{j}{idx}"
                            data["values"] = model.encode(dataset[disease_name][j][question]).tolist()
                            data["metadata"] = {}
                            data["metadata"]["title"] = question
                            data["metadata"]["content"] = dataset[disease_name][j][question]
                            data["metadata"]["source"] = dataset[disease_name][j]["source"]
                            
                            #index pinecone
                            try:
                                index.upsert(
                                    [data]
                                )
                            except Exception as e:
                                print("Error while indexing")
                        
                    print(f"{disease_name} indexed!")
                print("___")    
if __name__ == "__main__":
    main()

dataset_b.json reading...
Babesiosis indexed!...
Baby Acne indexed!...
dataset_a.json reading...
AA Amyloidosis indexed!...
Abdominal Adhesions indexed!...
